WORK WITH TERMINAL

- Source oneAPI environment variables: $source /opt/intel/oneapi/setvars.sh <br>
- Create conda env: $conda create --name intel-tensorflow --clone tensorflow <br>
- Activate the created conda env: $source activate intel-tensorflow <br>
- Install the extra needed package: (intel-tensorflow) $pip install cxxfilt matplotlib ipykernel psutil pandas gitpython <br>
- Deactivate conda env: (intel-tensorflow)$conda deactivate <br>
- Register the kernel to Jupyter NB: $~/.conda/envs/intel-tensorflow/bin/python  -m ipykernel install --user --name=intel-tensorflow

WORK WITH JUPYTER NOTEBOOK

In [1]:
import opendatasets as od

In [2]:
dataset= "https://www.kaggle.com/datasets/gpiosenka/100-bird-species/download?datasetVersionNumber=62"

In [ ]:
od.download(dataset)

In [ ]:
import gradio as gr

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib

data_dir = 'Data'
data_dir = pathlib.Path(data_dir)

In [ ]:
import pathlib

data_dir = 'Data/train'
data_dir = pathlib.Path(data_dir)

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
num_classes = 500

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
def predict_image(img):
  img_4d=img.reshape(-1,180,180,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(500)}

In [ ]:
image = gr.inputs.Image(shape=(180,180))
label = gr.outputs.Label(num_top_classes=500)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(share='True')